# Я, похоже, фундаментально ошибся - 

надо не юзеров искать, а посмотреть статистику правильных объектов...

## ВОТ И СРАВНЮ



In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
def func(df):
    good1 = df.groupby("itemId")['Xmin', 'Ymin'].min()
    good2 = df.groupby("itemId")['Xmax', 'Ymax', 'Xmin_true',	'Ymin_true',	'Xmax_true',	'Ymax_true'].max()
    good = pd.concat([good1, good2], axis=1)
    good['iou'] = good.apply(intersection_over_union, axis=1)
    #print(good['iou'].mean())
    return good['iou'].mean()

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)
maxbox = pd.read_csv("drive/My Drive/sna9/r4.csv", header=None).rename(columns={0: "itemId", 1: "Xmin", 2:'Ymin', 3:'Xmax', 4:'Ymax'})

In [0]:
votes.head()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D,iou,best,Lt,Ht,Dt,Lt/Ht
0,1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729,0.613974,1,166,162,231.94827,1.024691
1,1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347,0.079069,0,166,162,231.94827,1.024691
2,1694,6132,63,648,272,830,47,650,213,812,0.989526,256,4,3481,324,1,1,0,1,209.0,182.0,782.0,38038.0,1.148352,277.137150,0.598080,1,166,162,231.94827,1.024691
3,916,6132,111,637,253,831,47,650,213,812,0.983958,4096,169,1600,361,0,0,1,1,142.0,194.0,672.0,27548.0,0.731959,240.416306,0.435805,0,166,162,231.94827,1.024691
4,1621,6132,60,651,242,819,47,650,213,812,0.997269,169,1,841,49,1,1,1,1,182.0,168.0,700.0,30576.0,1.083333,247.685284,0.750206,1,166,162,231.94827,1.024691


In [0]:
# right parameters
votes['Lt'] = votes.Xmax_true - votes.Xmin_true
votes['Ht'] = votes.Ymax_true - votes.Ymin_true
votes['Dt'] = np.sqrt(votes.Lt**2 + votes.Ht**2)
votes['Lt/Ht'] = votes.Lt / votes.Ht
votes['St'] = votes.Lt * votes.Ht

In [0]:
votes[['L', 'H', 'Lt', 'Ht']].describe()

,L,H,Lt,Ht
count,5190.000000,5190.000000,5190.000000,5190.000000
mean,314.322736,271.363969,380.187283,317.913680
std,223.295951,190.264457,247.086432,212.031726
min,2.000000,2.000000,6.000000,4.000000
25%,133.000000,114.000000,175.000000,142.000000
50%,276.000000,242.000000,349.500000,287.000000
75%,459.000000,401.000000,565.000000,462.000000
max,1021.000000,940.000000,1025.000000,997.000000


In [0]:
votes = votes[((votes.L>=2) & (votes.H>=2) & (votes.L<=1025) & (votes.H<=997) & (votes.D<=1398) & \
               (votes.D>=3.6) & (votes['L/H']<=3.098901) & (votes['L/H']>=0.398773) )]
votes.shape

(5190, 32)

In [0]:
func(votes)

0.5819833699453295


## GREAT EXPERIMENT

In [0]:
mb = maxbox.set_index('itemId')

In [0]:
mb.head()

In [0]:
for i in range(29, 630, 18):
    mb.Ymin.iloc[i] -= 1
    mb.Ymax.iloc[i] += 1
    #mb.Xmin.iloc[i] -= 1
    mb.Xmax.iloc[i] += 1

In [0]:
mb.to_csv("drive/My Drive/sna9/foosol_085.csv", header=None)

In [0]:
mb.to_csv("drive/My Drive/sna9/mbup4.csv", header=None)

In [0]:
# select good users:
# by max iou
limit = 0.475980 # 50%     
goods = votes[votes.iou > limit]
goods.shape  # (2678, 27)
len(goods.itemId.unique()) # 891 vs 941
len(goods.userId.unique()) # 996 vs 1335
func(goods) # 0.6156633235642854 But items NOT ALL!!!

0.6162643314578299


In [0]:
# bads (vice versa)
limit = 0.3
bads = votes[votes.iou < limit]
bads.shape  # 
goods = votes.drop(bads.index)
goods.shape # (4699, 27)
len(goods.itemId.unique()) # 891 vs 941
len(goods.userId.unique()) # 996 vs 1335
func(goods) # 0.5830294390080151

0.5860265453209792


## ok. On test:

In [0]:
votest = votest[((votest.L>=6) & (votest.H>=4) & (votest.L<=1025) & (votest.H<=997))] # & (votest.D<=1398) & (votest.D>=3.6) & (votest['L/H']<=3.098901) & (votest['L/H']>=0.398773))]
votest.shape

(3463, 12)

In [0]:
tgood1 = votest.groupby("itemId")['Xmin', 'Ymin'].min()
tgood2 = votest.groupby("itemId")['Xmax', 'Ymax'].max()
tgood = pd.concat([tgood1, tgood2], axis=1)

diff = list(set(maxbox.itemId.unique()) - set(tgood.index.unique()))
add = maxbox[maxbox.itemId.isin(diff)].set_index('itemId')
tgood = pd.concat([tgood, add]).sort_index()

tgood.to_csv("drive/My Drive/sna9/i8.csv", header=None)

In [0]:
len(tgood.index.unique())

627

In [0]:
tgood.to_csv("drive/My Drive/sna9/y6.csv", header=None)

In [0]:
# говнюки
for i in range (30, 50): #0.3, 0.35, 0.4, 0.45, 0.475]:
    shiters = votes[votes.iou < i/100]
    print(i)
    shiters.iou.mean()
    shiters.head()
    func(votes.drop(shiters.index))
    func(votes)
    print(votes.drop(shiters.index).shape)
    print(len(votes.drop(shiters.index).itemId.unique()))

In [0]:
#dfs = zones(votest)
rate = dfs[7:]
df0 = pd.concat(rate)
df0.shape

(4705, 12)

ЛАБА НЕХАЕВА

In [0]:
""" параметры графиков """
plt.style.use('ggplot')
plt.rcParams['font.fantasy'] = 'Arial', 'Times New Roman', 'Tahoma', 'Comic Sans MS', 'Courier'
plt.rcParams['font.family'] = 'fantasy'